# Examples of msms_wrapper use

This is to demonstrate use of a Python wrapper for the `msms` command-line program developed by Dr. Michel F. Sanner. The wrapper and msms can be used to compute triangulated solvent-excluded surfaces of molecules, as well as their solvent-accessible surface (SAS), solvent excluded surface (SES), and molecular volume.

---------

## Initial Setup

First some disclosures and addressing a licensing issue....

Note: While the Riniker Lab has put this wrapper under the MIT licence, `msms` is not. The msms_wrapper & effort to make it work with MyBinder-served Jupyter sessions are independent projects, and not affiliated with the `msms` program. You can obtain `msms` from https://ccsb.scripps.edu/msms/. For more information about the algorithm, see:

Sanner, M. F., Olson A.J. & Spehner, J.-C. (1996). Reduced Surface: An Efficient Way to Compute Molecular Surfaces. Biopolymers 38:305-320.

If you qualify for the license for detailed there, you can run the next cell in this session to get and install msmms:

In [ ]:
msms_soft_zipped = "msms_i86_64Linux2_2.6.1.tar.gz"
!mkdir -p ~/.local/bin
%cd -q ~/.local/bin
!curl -OL https://ccsb.scripps.edu/msms/download/933/{msms_soft_zipped}
!tar xzf {msms_soft_zipped}
msms_soft_fn = msms_soft_zipped.replace("_i",".x").replace("Linux2_2","Linux2.2")[:-7]
!ln -s ~/.local/bin/{msms_soft_fn} msms
%cd -q ~
print("***`msms` has been installed and an alias set in the system path that matches what the `msms_wrapper.py` script expects.***")

With the msms software installed, run the cell below to complete the initial set up in order to run the code in the rest of the notebook.

In [ ]:
from io import BytesIO
import os

import numpy as np
import msms.wrapper as msms
import requests

from rdkit import Chem

# Check the installation

## Is msms available?
msms needs to be in the PATH. We can check this using `msms_available`. If msms does not exist (or if we delete the PATH), it will return False.

In [ ]:
print("Can find msms?", msms.msms_available())

In [ ]:
path = os.environ['PATH']
os.environ['PATH'] = ""
print("With PATH set to empty:")
print("Can find msms?", msms.msms_available())
os.environ['PATH'] = path

## Get help

In [ ]:
print(msms.help())

That last cell actually had contributions from both the msms software and the wrapper, and so if that cell above works, the rest of the noteboook should work, for the most part.

# Get a structure
**We need:**
* coordinates (in this case, from the RCSB pdb)
* radii (in this case, using the mBondi2 definition as used in the Ambertools)

In [ ]:
response = requests.get("https://files.rcsb.org/ligands/download/5P8_model.sdf")
lorlatinib = next(Chem.ForwardSDMolSupplier(BytesIO(response.content)))

In [ ]:
points = lorlatinib.GetConformer(0).GetPositions()
points -= points.mean(0)

In [ ]:
MBONDI2_RADII = {
    "C": 1.7,
    "N": 1.55,
    "O": 1.8,
    "Cl": 1.5,
    "Si": 2.1,
    "P": 1.85,
    "S": 1.8,
    "Br": 1.7,
}

def get_mbondi2_radii(mol):
    """Return the mBondi2 radii of a mol as a list"""
    periodic_table = Chem.GetPeriodicTable()
    out = []
    for i_atom, atom in enumerate(mol.GetAtoms()):
        elem = periodic_table.GetElementSymbol(atom.GetAtomicNum())
        if elem in MBONDI2_RADII:
            radius = MBONDI2_RADII[elem]
        elif elem == "H":
            bonded = atom.GetNeighbors()[0]
            bonded_elem = periodic_table.GetElementSymbol(bonded.GetAtomicNum())
            if bonded_elem == "N":
                radius = 1.3
            else:
                radius = 1.2
        else:
            radius = 1.5
        out.append(radius)
    return np.array(out)

In [ ]:
radii = get_mbondi2_radii(lorlatinib)

# Run MSMS
## Usage
* Pairs of arguments can be added as `kwargs`, usually like `density=2.0` or `probe_radius=1.0`
* Further msms arguments can be added as `*args`, like `"-free_vertices"`
## Output format
* msms_out.vertices contains all information on vertices (position, normals, type etc.)
* Best split it into several numpy arrays

In [ ]:
msms_out = msms.run_msms(points, radii, density=5.0, probe_radius=1)
verts = msms_out.get_vertex_positions()
normals = msms_out.get_vertex_normals()
faces = msms_out.get_face_indices()

# Minimal usage example

Use `msms` to compute the surface area of a unit sphere.

* The SES and SAS are analyical.
* The volume is numerical. In the case of a single sphere, it is always too small, but converges with high density.

In [ ]:
xyz = [[0., 0., 0.]]
radii = [1.]
print('expected SES', 4*np.pi)
print('expected SAS', 4*np.pi * 2.5**2) # 2.5 = radius + probe_radius
print('expected volume', 4/3*np.pi)
msms.run_msms(xyz, radii, density=2).extract_ses_sas_vol()

# Visualize
(NOTE THIS SECTION FROM THE ORIGINAL [msms_wrapper/examples.ipynb](https://nbviewer.org/github/rinikerlab/msms_wrapper/blob/main/examples.ipynb) WON'T WORK AS WRITTEN CURRENTLY IN SESSIONS LAUNCHED VIA MyBinder. This is because a the system running MyBinder involves a remote JupyterHub and as spelled out [here in the section 'Remote JupyterHubs'](https://tutorial.pyvista.org/tutorial/00_jupyter/index.html#remote-jupyterhubs) pyvista is incompatible. At this point, I don't know enough about what type of data is involved to see how to connect it. But I think it should be possible given I know ipyvolume works in Jupyter and https://github.com/InsightSoftwareConsortium/itk-jupyter-widgets .I note that VTK should work in MyBinder as I've used it before, see even the launch badge [here](https://github.com/trungleduc/jupyterview).)

If `pyvista` (or a similar package) is installed, the surface can be visualized in a Jupyter Notebook

Note: this sometimes crashes when using `Run all cells`

In [ ]:
import pyvista

In [ ]:
def for_pyvista(arr):
    out = []
    for row in arr:
        out.append(len(row))
        out.extend(row)
    return out

In [ ]:
surf = pyvista.PolyData(verts, faces=for_pyvista(faces))
surf.plot()